<a href="https://colab.research.google.com/github/JJOK97/core-project-study/blob/js%2Ftext-mining/jinseok/textmining%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%ED%8F%89%EC%A0%90%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 목표 설정
- 네이버 영화 리뷰 데이터셋을 이용해서 긍정/부정을 판별하는 분류기를 만들어보자.
- TF-IDF 방법을 이용해서 토큰화를 진행해보자.
- Konlpy 한국어 형태소 분석기를 이용해보자.
- 단어별 긍/부정 정보를 시각화


In [ ]:
# 내 경로 설정
!pwd

from google.colab  import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


In [ ]:

#소스코드가 있는 경로로 위치를 변경
%cd /content/drive/MyDrive/Colab Notebooks

#필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/ratings_train.csv', encoding = 'utf-8')

train.head()


/content/drive/MyDrive/Colab Notebooks


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
#test 데이터 불러오기
test = pd.read_csv('./data/ratings_test.csv', encoding = 'utf-8')

test.head()

#train df의 대한 정보를 확인해보자

train.info()

#test df도 확인
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [ ]:
#결측치 확인
train[train['document'].isnull()]

# .dropna : 결측치가 있는 행을 삭제해주는 함수
train.dropna(inplace = True)
test.dropna(inplace = True)

train.info() #결측치 꼭 없애기!

test.info()

train #X(문제)는 document, Y(정답)는 label --> 지도학습


<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
#학습데이터의 정답데이터
train['label']

#학습데이터의 문제데이터
train['document']

#평가데이터의 문제데이터
test['document']

,document
0,굳 ㅋ
1,GDNTOPCLASSINTHECLUB
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네


In [ ]:
#train/ test 데이터셋에서 문제데이터/ 정답데이터 지정

#x : 문제
#y : 정답
#train : 훈련
#test : 평가(테스트)

#문제 데이터
x_train = train['document']
x_test = test['document']

#정답 데이터
y_train = train['label']
y_test = test['label']

x_train.shape, x_test.shape,y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))


### 토큰화
**vectorizer**
- vectorizer는 대상을 벡터화시켜주는 도구
- 벡터화 : 텍스트나 이미지처럼 '비수치적'인 데이터를 숫자 배열(벡터)로 바꾸는 과정
- 기계 학습 분야에서 주로 텍스트 데이터를 분석하기 위해서
- 여러 머신러닝/ 딥러닝 모델을 학습시키기 위해서는 텍스트나 다른 형태의 데이터를 벡터화/수치화할 필요가 있다.

### BOW(Bag of Word) - 단어가방
- 문서를 벡터로 변환하는 가장 기본적인 방법
- 문서 내의 모든 단어를 모아서 가방 안에 넣고, 어떤 단어가
들어 있는지 리스트/딕셔너리 형태로 '순서-숫자'로 나타내는 방법

In [ ]:
# TF-IDF 기법 사용
from sklearn.feature_extraction.text import TfidfVectorizer


x_train[0:3]

#TF-IDF 벡터라이저 사용

#객체 생성
tfidf = TfidfVectorizer()

#단어 사전 구축
tfidf.fit(x_train[0:3])


#분리된 단어로 만든 단어사전
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [ ]:



temp = TfidfVectorizer() #벡터화 객체화

sugar = '설탕국수를냠냠쩝쩝너무맛있게먹었다냠냠쩝쩝'
boy = 'i am a boy'
boy2 = '나는 소년이다.'

#단어사전 구축
temp.fit([sugar, boy, boy2])

temp.vocabulary_

{'설탕국수를냠냠쩝쩝너무맛있게먹었다냠냠쩝쩝': 3, 'am': 0, 'boy': 1, '나는': 2, '소년이다': 4}

- TF-IDF Vectorizer는 기본적으로 어절 단위(문장을 이루는 토막)로 끊어주기 때문에 한국어 특성상 파악이 어렵다.

- 더 작은 단위인 형태소별로 나눠서 파악해야 한다.

## 한국어 형태소 분석기 사용
- 한국어 특성에 맞춰 개발된 분석기 사용


In [ ]:
#라이브러리 설치
!pip install konlpy

#형태소 분석기 불러오기
from konlpy.tag import Kkma

# ## Kkma(꼬꼬마)
# - 서울대학교 IDS연구실 개발


# 꼬꼬마 객체 생성
kkma = Kkma()

sugar

kkma.nouns(sugar)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 30.9 MB/s eta 0:00:00


['설탕', '국수']

### Kkma에서 사용하는 대표적 함수
- nouns() : 명사 추출 함수
- morphs() : 형태소 추출
- pos() : 형태소 추출 + 품사 태그





In [ ]:
# tfidf + Kkma 명사 추출기 연결해서 사용해보기
# tf-idf Vectorizer에 연결하기 위해서, 명사 추출기를 함수화
def myTokenizer(text):
  return kkma.nouns(text)

x_train[0:3]

# tfidf = TfidfVectorizer()

tfidf_nouns = TfidfVectorizer(tokenizer=myTokenizer)

# 적용/학습
tfidf_nouns.fit(x_train[0:3])

# 단어사전 확인
tfidf_nouns.vocabulary_

x_train[0]

# 형태소 추출
kkma.morphs(x_train[0])

# 품사 태그 종류 출력해보기
kkma.tagset

# 우리가 지금 진행하고 있는 분석 감성 분류 분석(긍정 / 부정)
# 감성분석에서는 동사 / 형용사 / 보통명사를 많이 사용한다.

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [ ]:
data = '러시아 월드컴에서 콩국수를 먹던데 굉장히 신기하고 나도 콩국수가 먹고 싶어졌다'

kkma.pos(data)

# 데잍터프레임화 시켜보기
df = pd.DataFrame(kkma.pos(data), columns=['morphs', 'tag'])
df


# 인덱스를 tag로 지정해보자.
df.set_index('tag', inplace=True)
df




,morphs
tag,
NNG,러시아
NNP,월드
NNP,컴
JKM,에서
NNG,콩국수
JKO,를
VV,먹
ECD,던데
MAG,굉장히


In [ ]:
# 문장데이터에 우리가 만들어준 토크나이저 적용해보자.

# 적용 준비 단계 - 함수화
def myTokenizer2(text):
  # 우리가 필요한 품사만 뽑아보자
  d = pd.DataFrame(kkma.pos(text), columns=['morphs', 'tag'])

  # 인덱스를 태그로 지정까지
  d.set_index('tag', inplace=True)

  # 동사, 형용사, 보통명사만 뽑아서 데이터 프레임을 만들어보자
  if ('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index):
    return d.loc[d.index.isin(['W', 'VA','NNG'])]
  else:
    return []

In [ ]:
# TF-IDF Vectorizer에 연결
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer2)

# 적용/학습
tfidf_pos.fit(x_train[0:3])

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x79a8af6e4fe0>)

In [37]:
tfidf_final = TfidfVectorizer(tokenizer=myTokenizer2)

# 10000개의 데이터를 이용해서 단어사전 구축을 진행
tfidf_final.fit(x_train[0:1000])

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x79a8af6e4fe0>)

In [ ]:
# .transform : 텍스트데이터를 벡터화
# 실제 문장을 잘게 쪼개서 수치값으로 바꿔주자

x_tarin_final = tfi